In [1]:
import pandas as pd

from src.visualize import draw_splines_on_circular_numbers

In [ ]:
# Read from CSV
df = pd.read_csv("data/complete_sets_n42.csv")

# Get name of last column
last_column_name = df.columns[-1]
n = int(last_column_name.split(" ")[-1]) * 2
n

42

In [10]:
# Clear images directory using pathlib
from pathlib import Path

images_dir = Path("images")
if images_dir.exists() and images_dir.is_dir():
    for file in images_dir.iterdir():
        if file.is_file():
            file.unlink()


# Iterate through the rows to process one row at a time
for index, row in df.iterrows():
    # Extract pairs from the row (assuming pairs are in the format "a,b"). Skip the first column which has Pair No.
    pairs = []
    for col in df.columns[1:]:
        pair_str = row[col]
        if pd.isna(pair_str):
            continue  # Skip if the cell is empty
        a, b = map(int, pair_str.split("-"))
        pairs.append((a, b))

    draw_splines_on_circular_numbers(n, pairs, f"images/n{n}_{index}.png")


Image with splines saved to images/n42_0.png
Image with splines saved to images/n42_1.png
Image with splines saved to images/n42_2.png
Image with splines saved to images/n42_3.png
Image with splines saved to images/n42_4.png
Image with splines saved to images/n42_5.png
Image with splines saved to images/n42_6.png
Image with splines saved to images/n42_7.png
Image with splines saved to images/n42_8.png
Image with splines saved to images/n42_9.png
Image with splines saved to images/n42_10.png
Image with splines saved to images/n42_11.png
Image with splines saved to images/n42_12.png
Image with splines saved to images/n42_13.png
Image with splines saved to images/n42_14.png
Image with splines saved to images/n42_15.png
Image with splines saved to images/n42_16.png
Image with splines saved to images/n42_17.png
Image with splines saved to images/n42_18.png
Image with splines saved to images/n42_19.png
Image with splines saved to images/n42_20.png
Image with splines saved to images/n42_21.pn

In [13]:
from video_maker import create_video_with_transitions

# # # Basic usage
# create_video_from_images(
#     image_folder="images/",
#     output_video=f"video/n{n}.mp4",
#     fps=30,
#     frames_per_image=10,  # 1 second per image
# )


# Basic usage - smooth transitions between images
create_video_with_transitions(
    image_folder="images/",
    output_video=f"video/n{n}.mp4",
    fps=30,
    frames_per_image=30,
    transition_frames=15,  # 0.5 second crossfade transitions
)

Found 2603 images
Video settings: 30 FPS, 30 frames per image, 15 transition frames
Total video duration: 3904.00 seconds (65.07 minutes)
Video dimensions: 1785x1785
Processing image 1/2603: n42_0.png
Processing image 2/2603: n42_1.png
Processing image 3/2603: n42_10.png
Processing image 4/2603: n42_100.png
Processing image 5/2603: n42_1000.png
Processing image 6/2603: n42_1001.png
Processing image 7/2603: n42_1002.png
Processing image 8/2603: n42_1003.png
Processing image 9/2603: n42_1004.png
Processing image 10/2603: n42_1005.png
Processing image 11/2603: n42_1006.png
Processing image 12/2603: n42_1007.png
Processing image 13/2603: n42_1008.png
Processing image 14/2603: n42_1009.png
Processing image 15/2603: n42_101.png
Processing image 16/2603: n42_1010.png
Processing image 17/2603: n42_1011.png
Processing image 18/2603: n42_1012.png
Processing image 19/2603: n42_1013.png
Processing image 20/2603: n42_1014.png
Processing image 21/2603: n42_1015.png
Processing image 22/2603: n42_1016

'video/n42.mp4'